In [21]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

In [20]:
url="https://www.magicbricks.com/property-for-sale/residential-real-estate?&proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Residential-House,Villa&cityName=Mumbai"
pages=1500
header={"user-agent":"Mozilla/4.0 (compatible; MSIE 6.0; Windows 98) XX"}
soup=BeautifulSoup(requests.get(url,headers=header).text)
property_links=[]
for i in soup.find_all('div',class_=lambda x: x and x.startswith('m-srp-card SRCard')):
    property_links.append(re.findall("https://www.magicbricks.com/.+",i.get("data-code"))[0].split(",")[0].strip("'"))
index=np.array(range(pages))*30
urls=[f"https://www.magicbricks.com/mbsearch/propertySearch.html?propertyType_new=10002_10003_10021_10022,10001_10017&postedSince=-1&city=4320&searchType=1&propertyType=10002,10003,10021,10022,10001,10017&disWeb=Y&pType=10002,10003,10021,10022,10001,10017&category=S&cusImgCount=0&groupstart={groupstart}&offset=0&maxOffset=18&attractiveIds=&page=2&ltrIds=54308069,51546621&preCompiledProp=&excludePropIds=&addpropertyDataSet="
      for groupstart in index]

In [21]:
for url in tqdm(urls):
    header={"user-agent":"Mozilla/4.0 (compatible; MSIE 6.0; Windows 98) XX"}
    soup=BeautifulSoup(requests.get(url,headers=header).text)
    for i in soup.find_all('div',class_=lambda x: x and x.startswith('m-srp-card SRCard')):
        try:
            property_links.append(re.findall("https://www.magicbricks.com/.+",i.get("data-code"))[0].split(",")[0].strip("'"))
        except:
            print("none")
    if len(property_links) >40000:
        break

 89%|█████████████████████████████████████████████████████████████████████▎        | 1332/1500 [31:22<03:57,  1.41s/it]


In [23]:
property_links

['https://www.magicbricks.com/propertyDetails/2-BHK-629-Sq-ft-Multistorey-Apartment-FOR-Sale-Chembur-in-Mumbai&id=4d423534333038303639?sem=Y&semsrc=CPLADS_1_PDP_Existing_Web&slot=1',
 'https://www.magicbricks.com/propertyDetails/2-BHK-631-Sq-ft-Multistorey-Apartment-FOR-Sale-Bhandup-West-in-Mumbai&id=4d423533303034373239',
 'https://www.magicbricks.com/propertyDetails/3-BHK-810-Sq-ft-Multistorey-Apartment-FOR-Sale-Sewri-in-Mumbai&id=4d423534333635323533',
 'https://www.magicbricks.com/propertyDetails/3-BHK-974-Sq-ft-Multistorey-Apartment-FOR-Sale-Chembur-in-Mumbai&id=4d423534333038303937',
 'https://www.magicbricks.com/propertyDetails/1-BHK-470-Sq-ft-Multistorey-Apartment-FOR-Sale-Ghatkopar-West-in-Mumbai&id=4d423533343631383237',
 'https://www.magicbricks.com/propertyDetails/2-BHK-710-Sq-ft-Multistorey-Apartment-FOR-Sale-Goregaon-East-in-Mumbai&id=4d423534313730333237',
 'https://www.magicbricks.com/propertyDetails/1-BHK-330-Sq-ft-Multistorey-Apartment-FOR-Sale-Mira-Road-East-in-Mumba

In [24]:
links_data=pd.DataFrame(property_links,columns=["property_links"])
links_data=pd.DataFrame(links_data.property_links.unique(),columns=["property_links"])
links_data.to_csv("mumbai_links.csv",index=False)

In [29]:
links_data.property_links.unique()

array(['https://www.magicbricks.com/propertyDetails/2-BHK-629-Sq-ft-Multistorey-Apartment-FOR-Sale-Chembur-in-Mumbai&id=4d423534333038303639?sem=Y&semsrc=CPLADS_1_PDP_Existing_Web&slot=1',
       'https://www.magicbricks.com/propertyDetails/2-BHK-631-Sq-ft-Multistorey-Apartment-FOR-Sale-Bhandup-West-in-Mumbai&id=4d423533303034373239',
       'https://www.magicbricks.com/propertyDetails/3-BHK-810-Sq-ft-Multistorey-Apartment-FOR-Sale-Sewri-in-Mumbai&id=4d423534333635323533',
       ...,
       'https://www.magicbricks.com/propertyDetails/3-BHK-1390-Sq-ft-Multistorey-Apartment-FOR-Sale-Dadar-West-in-Mumbai&id=4d423535333739373939',
       'https://www.magicbricks.com/propertyDetails/3-BHK-1450-Sq-ft-Multistorey-Apartment-FOR-Sale-Malad-East-in-Mumbai&id=4d423437333538363733',
       'https://www.magicbricks.com/propertyDetails/2-BHK-617-Sq-ft-Multistorey-Apartment-FOR-Sale-Mulund-West-in-Mumbai&id=4d423535323434383237'],
      dtype=object)

In [33]:
def new_func(links_data,no):
    price=[]
    Address=[]
    area=[]
    carea=[]
    latitude=[]
    longitude=[]
    Bedrooms=[]
    Bathrooms=[]
    Balcony=[]
    Status=[]
    neworold=[]
    parking=[]
    Furnished_status=[]
    Lift=[]
    Landmarks=[]
    type_of_building=[]
    desc=[]
    for url in tqdm(list(links_data.property_links)):
        header={"user-agent":"Mozilla/3.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36"}
        soup=BeautifulSoup(requests.get(url,headers=header).text)                  

        #price
        try:     
            temp=(soup.find("div",class_="p_price").get_text()).split()
            if temp[2]=="Lac":
                temp=float(temp[1])*100000
            elif temp[2]=="Cr":
                temp=float(temp[1])*10000000    
            price.append(str(temp))
        except:
            price.append("nil")
        if price[-1]=="nil":
            price.pop()
            continue
        #latitude and longitude
        try:
            long=soup.find_all("meta",itemprop="longitude")
            lat=soup.find_all("meta",itemprop="latitude")
            long=str(long[0])
            lat=str(lat[0])
            long=float(((long.split())[1].split("="))[1].replace('"',''))
            lat=float(((lat.split())[1].split("="))[1].replace('"',''))
            if lat< long:
                latitude.append(str(lat))
                longitude.append(str(long))
            else: 
                longitude.append(str(lat))
                latitude.append(str(long))
        except:
            longitude.append("nil")
            latitude.append("nil")

        #type of building
        try:
            for i in (re.sub("\t","",re.sub("\n","",soup.find("div",class_="propBhk").h1.span.get_text()))).split():
                if i =="flat" or i=="Apartment" or i=="apartment" or i=="Flat"  :
                    type_of_building.append("Flat")
                elif i == "House" or i=="house" or i=="Villa" or i=="villa":
                    type_of_building.append("Individual House")
        except:
             type_of_building.append("nil")

        #################################################################################################################
        p_title=[]
        pvalue=[]
        #scraping all the all the titles and their values from p_infoColumn block
        #then extracting required variabe values using title indexing
        for prop in soup.find_all("div",class_="p_infoColumn"):
            p_title.append(prop.find("div",class_="p_title").get_text().strip())
            pvalue.append(prop.find("div",class_="p_value").get_text().strip())

        #Bedrooms
        try:
            Bedrooms.append(re.search("\d+",pvalue[p_title.index("Bedrooms")]).group())
        except:
            Bedrooms.append("nil")

        #Bathrooms
        try:
            Bathrooms.append(pvalue[p_title.index("Bathrooms")])
        except:
            Bathrooms.append("nil")

        #Balcony
        try:
            Balcony.append(pvalue[p_title.index("Balcony")])
        except:
            try:
                Balcony.append(pvalue[p_title.index("Balconies")])
            except:
                Balcony.append("nil")

        #area
        try:
            area.append(re.search("\d+",pvalue[p_title.index("Super area")]).group())
        except:
            area.append("nil")    
        try:
            carea.append(re.search("\d+",pvalue[p_title.index("Carpet area")]).group())
        except:
            carea.append("nil")      

        #Status
        try:
            Status.append(pvalue[p_title.index("Status")])
        except:
            Status.append("nil")

        #neworold
        try:
            neworold.append(pvalue[p_title.index("Transaction type")])
        except:
            neworold.append("nil")


        #parking
        try:
            parking.append(pvalue[p_title.index("Car parking")])
        except:
            parking.append("nil")

        #Furnished status
        try:
            Furnished_status.append(pvalue[p_title.index("Furnished status")])
        except:
            Furnished_status.append("nil")
        #################################################################################################################

        desc_block=soup.find("div",class_="descriptionCont")

        #description
        desc.append(desc_block.div.get_text())

        p_title=[]
        pvalue=[]
        #scraping all the all the titles and their values from p_infoRow block
        #then extracting required variabe values using title indexing
        for prop in desc_block.find_all("div",class_="p_infoRow"):
            try:
                pvalue.append(prop.find("div",class_="p_value").get_text().strip())
            except:
                pvalue.append("nil")
            try:
                p_title.append(prop.find("div",class_="p_title").get_text().strip())
            except:
                p_title.append("nil")


        #Lift
        try:
            Lift.append(pvalue[p_title.index("Lift")])
        except:
            Lift.append("nil")


        #Address
        try:
            Address.append(pvalue[p_title.index("Address")])
        except:
            Address.append("nil")

        #Landmarks
        try:
            Landmarks.append(pvalue[p_title.index("Landmarks")])
        except:
            Landmarks.append("nil")
        #############################################################################################
        
    data=list(zip(price,Address,area,carea,latitude,longitude,Bedrooms,Bathrooms,Balcony,Status,neworold,parking,Furnished_status,Lift,Landmarks,type_of_building,desc))
    frame=pd.DataFrame(data)
    frame.columns=['price', 'Address', 'area','carpet area', 'latitude', 'longitude',
       'Bedrooms', 'Bathrooms', 'Balcony', 'Status',
       'neworold', 'parking', 'Furnished_status', 'Lift',
       'Landmarks','type_of_building', 'desc']
    frame.to_csv("mumbai"+no+".csv",index=False)

In [13]:
from multiprocessing import Process
si=(links_data.shape)[0]
a=links_data[0:int(si/4)]
b=links_data[int(si/4):2*int(si/4)]
c=links_data[2*int(si/4):3*int(si/4)]
d=links_data[3*int(si/4):(4*int(si/4))-1]

t1=Process(target=new_func,args=(a,"1"))
t2=Process(target=new_func,args=(b,"2"))
t3=Process(target=new_func,args=(c,"3"))
t4=Process(target=new_func,args=(d,"4"))
t1.start()
t2.start()
t3.start()
t4.start()


t1.join()
t2.join()
t3.join()
t4.join()

In [442]:
mumbai1=pd.read_csv("mumbai1.csv")
mumbai2=pd.read_csv("mumbai2.csv")
mumbai3=pd.read_csv("mumbai3.csv")
mumbai4=pd.read_csv("mumbai4.csv")

In [443]:
mumbai=pd.concat([mumbai1, mumbai2,mumbai3,mumbai4], axis=0, ignore_index=True)


In [444]:
mumbai=mumbai[mumbai.area!="nil"]
mumbai=mumbai[mumbai.latitude!="nil"]
mumbai=mumbai[mumbai.longitude!="nil"]
mumbai=mumbai[mumbai.Bedrooms!="nil"]
mumbai=mumbai[mumbai.Bathrooms!="nil"]
mumbai=mumbai[mumbai.price!="['Call', 'for', 'Price']"]
mumbai=mumbai[mumbai.neworold!='Rent\n\t\t\t\t\t\t\t\t\tGet Documents Verified']

In [445]:
mumbai.replace("nil",np.nan,inplace=True)
mumbai.replace("None",np.int64(0),inplace=True)
mumbai.replace("New Property\n\t\t\t\t\t\t\t\t\tGet Documents Verified","New Property",inplace=True)
mumbai.replace('Resale\n\t\t\t\t\t\t\t\t\tGet Documents Verified',"Resale",inplace=True)


In [446]:

mumbai.shape


(6438, 17)

In [447]:
temp=mumbai1

In [448]:
l=[]
for i in (mumbai.parking):
    num=0
    try:
        for j in i.split(" "):
            try:
                num=num+int(j)
            except:
                temp=0
    except:
        num=np.nan   
    l.append(num)
mumbai.parking=l    
            

        

In [449]:
mumbai=mumbai[mumbai.Balcony!='10-plus']

In [450]:
mumbai=mumbai[mumbai.Bathrooms!='> 10']
            

In [451]:
mumbai["carpet area"].isnull().sum()

2959

In [452]:
mumbai.area=(mumbai.area).astype(np.float64)
mumbai.latitude=(mumbai.latitude).astype(np.float64)
mumbai.longitude=(mumbai.longitude).astype(np.float64)
mumbai.Bedrooms=(mumbai.Bedrooms).astype(np.float64)
mumbai.Bathrooms=(mumbai.Bathrooms).astype(np.float64)
mumbai.Balcony=(mumbai.Balcony).astype(np.float64)
mumbai.parking=(mumbai.parking).astype(np.float64)
mumbai.Lift=(mumbai.Lift).astype(np.float64)
mumbai.price=(mumbai.price).astype(np.float64)
mumbai["carpet area"]=(mumbai["carpet area"]).astype(np.float64)

In [453]:
mumbai.describe()

,price,area,carpet area,latitude,longitude,Bedrooms,Bathrooms,Balcony,parking,Lift
count,6.427000e+03,6427.000000,3468.000000,6427.000000,6427.000000,6427.000000,6427.000000,2430.000000,2170.000000,1707.000000
mean,2.776905e+07,1168.396297,918.111303,19.141774,72.768324,2.454333,2.429127,2.157202,4.689401,2.472759
std,4.209583e+07,641.938227,493.167997,0.766241,2.846738,0.760787,0.783274,1.239189,35.642342,1.626546
min,9.000000e+05,10.000000,13.000000,0.000000,0.000000,2.000000,0.000000,1.000000,1.000000,0.000000
25%,1.300000e+07,830.000000,650.000000,19.081509,72.832372,2.000000,2.000000,1.000000,1.000000,2.000000
50%,1.900000e+07,1000.000000,760.000000,19.147240,72.857530,2.000000,2.000000,2.000000,1.000000,2.000000
75%,3.000000e+07,1300.000000,1010.500000,19.213390,72.884691,3.000000,3.000000,3.000000,2.000000,3.000000
max,2.000000e+09,8000.000000,7800.000000,28.552851,84.482620,10.000000,10.000000,10.000000,801.000000,10.000000


In [454]:
temp=mumbai.price/mumbai.area

In [455]:
mumbai["Price_sqft"]=temp

In [456]:
mumbai=mumbai[mumbai.Price_sqft<120000]

In [468]:
mumbai=mumbai[mumbai.area>500]
mumbai.drop(columns="carpet area",inplace=True)

In [478]:
mumbai=mumbai[mumbai.latitude>18.8]
mumbai=mumbai[mumbai.latitude<19.5]
mumbai=mumbai[mumbai.longitude>72.75]
mumbai=mumbai[mumbai.longitude<74]

In [479]:
mumbai.describe()

,price,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,parking,Lift,Price_sqft
count,6.255000e+03,6255.000000,6255.000000,6255.000000,6255.000000,6255.000000,2376.000000,2129.000000,1678.000000,6255.000000
mean,2.684187e+07,1185.625260,19.156772,72.868642,2.452278,2.425580,2.162879,4.662283,2.448153,21422.923601
std,2.788831e+07,636.973259,0.109447,0.058614,0.749534,0.762604,1.221899,35.780057,1.566455,12698.478665
min,1.500000e+06,503.000000,18.873713,72.754080,2.000000,0.000000,1.000000,1.000000,0.000000,1500.000000
25%,1.300000e+07,846.500000,19.081509,72.832930,2.000000,2.000000,1.000000,1.000000,2.000000,13709.934735
50%,1.900000e+07,1000.000000,19.147269,72.858019,2.000000,2.000000,2.000000,1.000000,2.000000,19285.714286
75%,3.000000e+07,1300.000000,19.213054,72.884691,3.000000,3.000000,3.000000,2.000000,3.000000,26500.000000
max,3.600000e+08,8000.000000,19.476239,73.197823,10.000000,10.000000,8.000000,801.000000,10.000000,119047.619048


In [480]:
mumbai.to_csv("mumbai.csv",index=False)